In [60]:
import torch
import torch.nn as nn
from transformers import AutoModel,AutoModelForMaskedLM,AutoTokenizer,EsmConfig
import os
import torch.nn.functional as F

class PLMinteract(nn.Module):
  def __init__(self,model_name,num_labels,embedding_size): 
    super(PLMinteract,self).__init__() 
    self.esm_mask = AutoModelForMaskedLM.from_pretrained(model_name) 
    self.embedding_size=embedding_size
    self.classifier = nn.Linear(embedding_size,1) # embedding_size 
    self.num_labels=num_labels

  def forward_test(self,features):
    embedding_output = self.esm_mask.base_model(**features, return_dict=True)
    embedding=embedding_output.last_hidden_state[:,0,:] #cls token
    embedding = F.relu(embedding)
    logits = self.classifier(embedding)
    logits=logits.view(-1)
    probability = torch.sigmoid(logits)
    return  probability

In [34]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [35]:
protein1 ="EGCVSNLMVCNLAYSGKLEELKESILADKSLATRTDQDSRTALHWACSAGHTEIVEFLLQLGVPVNDKDDAGWSPLHIAASAGRDEIVKALLGKGAQVNAVNQNGCTPLHYAASKNRHEIAVMLLEGGANPDAKDHYEATAMHRAAAKGNLKMIHILLYYKASTNIQDTEGNTPLHLACDEERVEEAKLLVSQGASIYIENKEEKTPLQVAKGGLGLILKRMVEG"
protein2= "MGQSQSGGHGPGGGKKDDKDKKKKYEPPVPTRVGKKKKKTKGPDAASKLPLVTPHTQCRLKLLKLERIKDYLLMEEEFIRNQEQMKPLEEKQEEERSKVDDLRGTPMSVGTLEEIIDDNHAIVSTSVGSEHYVSILSFVDKDLLEPGCSVLLNHKVHAVIGVLMDDTDPLVTVMKVEKAPQETYADIGGLDNQIQEIKESVELPLTHPEYYEEMGIKPPKGVILYGPPGTGKTLLAKAVANQTSATFLRVVGSELIQKYLGDGPKLVRELFRVAEEHAPSIVFIDEIDAIGTKRYDSNSGGEREIQRTMLELLNQLDGFDSRGDVKVIMATNRIETLDPALIRPGRIDRKIEFPLPDEKTKKRIFQIHTSRMTLADDVTLDDLIMAKDDLSGADIKAICTEAGLMALRERRMKVTNEDFKKSKENVLYKKQEGTPEGLYL"

In [58]:
# model_name= '/mnt/data/project0019/danliu/PPI/offline/esm2_t33_650M_UR50D'
# folder_huggingface_download='/mnt/data/project0019/danliu/PPI/checkpoint/650m/dscript/task6/2024-03-22_12-00-48-1mask_10classi_dscript_1plus/huggingface_model'

model_name= '/mnt/data/project0019/danliu/PPI/offline/esm2_t12_35M_UR50D'
folder_huggingface_download='/mnt/data/project0019/danliu/PPI/checkpoint/35m/dscript/task2/huggingface_model'

config = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name) 

Some weights of the model checkpoint at /mnt/data/project0019/danliu/PPI/offline/esm2_t12_35M_UR50D were not used when initializing EsmModel: ['contact_head.regression.weight', 'contact_head.regression.bias']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# embedding_size =1280
# PLMinter= PLMinteract(model_name, 1, config, embedding_size)

In [66]:
embedding_size =480
PLMinter= PLMinteract(model_name, 1, embedding_size)


ValueError: Unrecognized configuration class <class 'transformers.models.esm.configuration_esm.EsmConfig'> for this kind of AutoModel: AutoModelForMaskedLM.
Model type should be one of AlbertConfig, BartConfig, BertConfig, BigBirdConfig, CamembertConfig, ConvBertConfig, Data2VecTextConfig, DebertaConfig, DebertaV2Config, DistilBertConfig, ElectraConfig, ErnieConfig, FlaubertConfig, FNetConfig, FunnelConfig, IBertConfig, LayoutLMConfig, LongformerConfig, LukeConfig, MBartConfig, MegatronBertConfig, MobileBertConfig, MPNetConfig, MvpConfig, NezhaConfig, NystromformerConfig, PerceiverConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RoFormerConfig, SqueezeBertConfig, TapasConfig, Wav2Vec2Config, XLMConfig, XLMRobertaConfig, XLMRobertaXLConfig, YosoConfig.

In [68]:
PLMinter

PLMinteract(
  (esm_mask): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0): EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, elementwise_affin

In [67]:
load_model = torch.load(f"{folder_huggingface_download}/pytorch_model.bin")
PLMinter.load_state_dict(load_model)

RuntimeError: Error(s) in loading state_dict for PLMinteract:
	Missing key(s) in state_dict: "esm_mask.embeddings.position_ids", "esm_mask.embeddings.word_embeddings.weight", "esm_mask.embeddings.position_embeddings.weight", "esm_mask.encoder.layer.0.attention.self.query.weight", "esm_mask.encoder.layer.0.attention.self.query.bias", "esm_mask.encoder.layer.0.attention.self.key.weight", "esm_mask.encoder.layer.0.attention.self.key.bias", "esm_mask.encoder.layer.0.attention.self.value.weight", "esm_mask.encoder.layer.0.attention.self.value.bias", "esm_mask.encoder.layer.0.attention.self.rotary_embeddings.inv_freq", "esm_mask.encoder.layer.0.attention.output.dense.weight", "esm_mask.encoder.layer.0.attention.output.dense.bias", "esm_mask.encoder.layer.0.attention.LayerNorm.weight", "esm_mask.encoder.layer.0.attention.LayerNorm.bias", "esm_mask.encoder.layer.0.intermediate.dense.weight", "esm_mask.encoder.layer.0.intermediate.dense.bias", "esm_mask.encoder.layer.0.output.dense.weight", "es

In [63]:
texts=[protein1, protein2]
tokenized = tokenizer(*texts, padding=True, truncation='longest_first', return_tensors="pt", max_length=1603)       
tokenized = tokenized.to(DEVICE)

In [64]:
PLMinter.eval()
PLMinter.to(DEVICE)
with torch.no_grad():
    probability = PLMinter.forward_test(tokenized)
    print(probability.item())

0.4626026153564453


In [65]:
PLMinter

PLMinteract(
  (esm_mask): EsmModel(
    (embeddings): EsmEmbeddings(
      (word_embeddings): Embedding(33, 480, padding_idx=1)
      (dropout): Dropout(p=0.0, inplace=False)
      (position_embeddings): Embedding(1026, 480, padding_idx=1)
    )
    (encoder): EsmEncoder(
      (layer): ModuleList(
        (0): EsmLayer(
          (attention): EsmAttention(
            (self): EsmSelfAttention(
              (query): Linear(in_features=480, out_features=480, bias=True)
              (key): Linear(in_features=480, out_features=480, bias=True)
              (value): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (rotary_embeddings): RotaryEmbedding()
            )
            (output): EsmSelfOutput(
              (dense): Linear(in_features=480, out_features=480, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (LayerNorm): LayerNorm((480,), eps=1e-05, elementwise_affin